#$4n + \mathscr{O}(1)$ T Complexity Quantum-Classical Comparison Oracle

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="mailto: noureldinyosri@gmail.com">Noureldin Yosri</a></br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;May 2023

## Abstract
This short note reports an improved T gate complexity of $4n + \mathscr{O}(1)$ for the quantum oracle for comparing a quantum number to a classical number.  

## Introduction
The current optimal implementation of a reversible oracle for comparing two quantum numbers of $n$ qubits each is $8n + \mathscr{O}(1)$ and is given in the supplementary materials of [Berry et al., 2018](https://doi.org/10.1038/s41534-018-0071-5). Their implementation uses divide and conquer technique to create a binary tree of depth $\log{n}$ whose leafs are the qubits of then numbers with intermediate values stored in $\mathscr{O}(n)$ ancillas. Each non leaf node uses two CSWAP operations and the value of the comparison is computed from the root node using one Toffoli. 

When this decomposition is applied to the case where one input is a list of qubits and the the other is a classical number (i.e. a string of bits) this decomposition gives a $6n + \mathscr{O}(1)$ T complexity since at the level right above the leafs half of the CSWAP operations collapse to either identity or SWAP, thus the number of CSWAPs becomes $\approx \frac{n}{2} + 2 \times \frac{n}{2} = \frac{3n}{2}$, leadining to $4 \times \frac{3n}{2} = 6n$ T operations.

In this paper we provide a different implementation with $4n + \mathscr{O}(1)$ T operations count based on modeling the problem as a finite state machine of $2n$ states each having two transitions.


## Equality as a special case
Before we proceed to the comparison oracle we take a look at the equality oracle actually as a special case with a lower T complexity of only $4n + \mathscr{O}(1)$, as it can be implemented as an And operation. The And operation itself can be done using only $4n + \mathscr{O}(1)$ as per [Babbush et al., 2018](https://doi.org/10.1103%2Fphysrevx.8.041015) and [Craig Gidney](https://algassert.com/post/1903).

In [ ]:
def equality_oracle(A: Sequence[cirq.Qid], B: int, z: cirq.Qid) -> cirq.OP_TREE:
  # Returns a decomposition of the oracle O_B |A>z> = |A>|z^(A == B)> in only 4n T operations.
  bits = reversed([(B >> i)&1 for i in range(len(A))])
  
  ancilla = cirq.NamedQubit('and_result')
  yield AND(cv=bits).on(*A, ancilla)  # `ancilla` now has the result of equality. uses 4n T operations.
  
  yield cirq.CNOT(ancilla, z)  # update result qubit.
  
  yield AND(cv=bits, adjoint=True).on(*A, ancilla)  # `ancilla` now has the result of equality. uses only cliffords and measurements.

## The $ < \textit{ and } \leq$ oracles with $4n + \mathscr{O}(1)$ T gates

### Inspiration
We will only consider the comparison oracles for $<$ and $\leq$ since the $>$ and $\geq$ oracles would be the same by swapping the inputs.

Consider a classical algorithm for comparing two numbers $A \textit{ and } B$ of equal length $n$. The algorithm scans the numbers from left to right until it a index $i^*$ where they differ and returns $A_{i^*} < B_{i^*}$. This is how C/C++ [std::strcmp](https://en.cppreference.com/w/cpp/string/byte/strcmp) works. 

Implicitly this algorithm has $n + 3$ states $\{e_0, \ldots, e_n\} \cup \{L, R\}$ where being in the $e_k$ state means the prefixes of length $k$ are equal. with transitions being the states governed by:

\begin{equation}
\begin{split}
    e_k \rightarrow e_{k+1} \textit{ if } u_k = v_k \\
    e_k \rightarrow L \textit{ if } u_k < v_k \\
    e_k \rightarrow R \textit{ if } u_k > v_k \\
\end{split}
\end{equation}

When the result of comparison between individual indicies becomes probabilistic these states form a Markov decision process with three terminal states $\{L, e_n, R\}$. This gives us an inspiration for a new implementation.

### Algorithm
We start by allocating $n+1$ qubits representing the $e_0, \ldots, e_n$ states and then scan the qubit register and number from left to right.

if the current bit is zero then we only need to compute the $e_k \rightarrow e_{k+1}$ transition since the qubit can't be less than zero. otherwise we need to compute the transition as well as the $e_k → L$ transition.

In [ ]:
def less_than_oracle(A: Sequence[cirq.Qid], B: int, z: cirq.Qid) -> cirq.OP_TREE:
  # Returns a decomposition of the oracle O_B |A>z> = |A>|z^(A < B)> in only 4n T operations.
  bits = reversed([(B >> i)&1 for i in range(len(A))])

  adjoint = []

  es = cirq.LineQid.range(len(A), dimension=2)
  ek = es.pop(0)

  # Initially our belief is that the numbers are equal.
  yield cirq.X(ek)
  adjoint.append(cirq.X(ek))

  ancilla = cirq.LineQid.range(len(A), dimension=2)
  for q, b, ekp1 in zip(A, bits, ancilla):
    if b:
      yield cirq.X(q)
      adjoint.append(cirq.X(q))

      # Temporarily hold e_k and not q
      yield And().on(q, ek, ekp1)
      adjoint.append(And(adjoint=True).on(q, ek, ekp1))

      # e_{k+1} currently has are_equal so far and (q != b)
      # which is equivalent to: Is the current prefix of the qubits < the prefix of B and the previous prefix equal?
      yield cirq.CNOT(ekp1, target)

      yield cirq.CNOT(ek, ekp1)  # Now e_{k+1} has the prefix equality.
      adjoint.append(cirq.CNOT(ek, ekp1))
    else:
      # e_{k+1} = e_k and not q
      yield And(cv=[1, 0]).on(ek, q, ekp1)
      adjoint.append(And(cv=[1, 0], adjoint=True).on(ek, q, ekp1))
    
    ek = ekp1

  # For the <= oracle we need one extra clifford operation
  # yield cirq.CNOT(are_equal, z)

  yield from reversed(adjoint)


## Improving the constant
The implementation above has T complexity of exactly $4n$ since there are exactly $n$ And gates each uses $4$ Ts. Note however that the first of them is not actually needed since one of its inputs is in the $ |1> $ state so it collapses to a `cirq.CNOT`. This gives a T complexity of $4(n-1) = 4n - 4$.

## Drawback
The new decomposition improves the T complexity from $6n + \mathscr{O}(1)$ to $4n + \mathscr{O}(1)$ however it has linear depth rather than logarithmic depth and both circuits will have linear width.